<a href="https://colab.research.google.com/github/hlecuanda/jupyter-notebooks-of-all-kinds/blob/master/Sintesis_de_voz.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sintesis de voz.

Esta libreta demuestra el uso de las API de google para sintesis de voz a part ir de texto (Text to Speech, o bien TTS)





## instalación y pre-requisitos:

usaremos la libreria cliente de google asi como logzero para las bitacoras requeridas

In [ ]:
!pip install --upgrade google-cloud-texttospeech
!pip install --upgrade google-cloud-iam

también es necesario activar el API de sintesis de texto en un proyecto especifico. Daso que esta libreta esta iseñada para ser ejecutada en un ambiente de ejecución de google,aprovecharemos que estos ambientes nativos incluyen los clientes necesarios en la linea de comando.  

Adicionalmente esto quiere decir que nos ahorraremos el proceso de autenticacion, al usar las credeniales por defecto de aplicacion (ADC) que es una prestacion de la plataforma de google cloud.

Si deseas ejecutar esta libreta en otros ambientes, aqui es donde puedes agrear los pasos de autenticacion requeridos (Cliente de Oauth o alternativamente una cuenta de servicio registrada)

In [ ]:
from google.colab import auth
PROJECT_ID = "mezaops" # @param {type: "string"}
auth.authenticate_user(project_id=PROJECT_ID)

In [ ]:
!gcloud config set project {PROJECT_ID}
!gcloud services enable texttospeech.googleapis.com

## Procesamiento del texto

primerom cargamos un archivo de texto desde el sistema local (colab esta en el abiente de ejecución en google, que es una máquina virtual remota)

para mejores resultados, debe tratarse de texto simple, cada párafo completo debe estar en una sola linea individual, y no debe haber espacios entre las lineas. La razon e esto, es que el sintentizador de voz consiera toa la linea para generar los sonidos, si guardando los margenes, rompes un enunciado en dos mas cortos, el audio contendrá una pausa extraña y en ocasiones puede cambiar el significado de las frases.


In [ ]:
from google.colab import files

uploaded = files.upload()

def msg(name):
    print(f'User uploaded file "{name}" with length {len(uploaded[name])} bytes')

_ = [msg(name) for name in uploaded.keys()]

filenames = list(uploaded)

In [ ]:
with open(filenames[0], "rt", encoding="utf-8", newline=None) as textfile:
    rawtextarray = textfile.readlines()

textarray = [line.strip() for line in rawtextarray]

print(textarray)

In [ ]:
from google.cloud import texttospeech

# Crear una instancia del cliente API de Síntesis de voz
client = texttospeech.TextToSpeechClient()


# Configurar la peticiónn al APIm con lenguaje, género de voz y el tipo
# de archivo a generad
voice = texttospeech.VoiceSelectionParams(
    language_code="en-US", ssml_gender=texttospeech.SsmlVoiceGender.NEUTRAL
)
audio_config = texttospeech.AudioConfig(
    audio_encoding=texttospeech.AudioEncoding.MP3
)

# Abrir el archivo de salida, en modo binario. cada párrafo manda una requisicion
# al api. al recibir la respuesta, es concadenada en el archivo hasta que se
# cierra al salir del bloque "with"
with open("output.mp3", "wb") as out:

    parano = 1
    for paragraph in textarray:
        synthesis_input = texttospeech.SynthesisInput(text=paragraph)
        response = client.synthesize_speech(
            input=synthesis_input, voice=voice, audio_config=audio_config
        )
        # Write the response to the output file.
        out.write(response.audio_content)
        print(f"párrafo {parano}/{len(textarray)} que inicia en {paragraph[0:15]}... sintetizado")

print("Listo, el resultado está en output.mp3")
